![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&file=BigQuery+-+Columns.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/new/BigQuery%20-%20Columns.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Fnew%2FBigQuery%2520-%2520Columns.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/new/BigQuery%20-%20Columns.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/new/BigQuery%20-%20Columns.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# BigQuery | Columns

Explaining BigQuery Column types by examples!

# BQ Explained | Columns
## Overview of BigQuery data types, conversion, and coercion

With a focus on these data types:
- STRUCT
- ARRAY (Mode = Repeated)
- JSON
- Nesting Records with each

---
**IN PROGRESS**

- Define columns of all possible types
  - Derived type, cast, coercion, define a specific type
- Show working with types, especially STRUCT, ARRAY
  - STRUCT
    - Struct is an unordered list
  - ARRAY
    - Array is an ordered list with type = array
  - JSON
    - direct and coverting to JSON
  - STRUCT of STRUCT
  - ARRAY of STRUCT
  - STRUCT of ARRAY

Thoughts to bring together:
- struct is a container, its like a dictionary in python
- array is list, just like in python

Resources:
- Data Types: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
  - Work with Arrays: https://cloud.google.com/bigquery/docs/reference/standard-sql/arrays
  - Work with JSON: https://cloud.google.com/bigquery/docs/reference/standard-sql/json-data
- Conversion: https://cloud.google.com/bigquery/docs/reference/standard-sql/conversion_rules


---

## Setup

inputs:

In [1]:
project_id = 'statmike-mlops-349915'
dataset_id = 'explained_columns'
table_id = 'example'

### Authenticate

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
!gcloud config list project

[core]
project = statmike-mlops-349915

Your active configuration is: [default]


### BigQuery Python Client

In [5]:
import json
from google.cloud import bigquery
bq = bigquery.Client(project = project_id)

## Create A Dataset

In [6]:
dataset = bigquery.Dataset(bq.dataset(dataset_id))
dataset.location = 'US'
dataset = bq.create_dataset(dataset, exists_ok=True)
print(f"The dataset {dataset.dataset_id} is in project {bq.project}")

The dataset explained_columns is in project statmike-mlops-349915


In [7]:
print(f'Review data in BigQuery:\nhttps://console.cloud.google.com/bigquery?project={project_id}')

Review data in BigQuery:
https://console.cloud.google.com/bigquery?project=statmike-mlops-349915


--- 
## Data Types and Modes

In [8]:
myJSON = json.dumps({"customer": [{"name": "Mike", "age": 35}, {"name": "Alice", "age": 36}]})

In [9]:
myJSON

'{"customer": [{"name": "Mike", "age": 35}, {"name": "Alice", "age": 36}]}'

In [10]:
query = f"""
CREATE OR REPLACE TABLE {project_id}.{dataset_id}.{table_id}_1 AS
WITH
  INPUTS AS (
    SELECT '{myJSON}' as inputJSON
  )
SELECT
  7 as myInt,
  CAST('stringhere' as BYTES) as myBytes,
  7.1 as myFloat,
  'string' as myString,
  True as myBool,
  ST_GEOGPOINT(81.3124, 29.9012) as myGeo,
  CURRENT_DATE() as myDate,
  CURRENT_DATETIME() as myDatetime,
  CURRENT_TIME() as myTime,
  CURRENT_TIMESTAMP() as myTimestamp,
  [1, 2, 3] as myArray,
  STRUCT(1 as col1, 2 as col2, 3 as col3) as myStruct,
  STRUCT(STRUCT('a' as col1, 'b' as col2, 'c' as col3) as myStructNestedA, STRUCT(1 as col1, 'b' as col2) as myStructNestedB) as myStructOfStruct,
  STRUCT([1, 2, 3] as myArrayNestedA, ['A','B','C','D'] as myArrayNestedB) as myStructOfArray,
  [STRUCT('a' as col1, 'B' as col2), STRUCT('1' as col1, 'b' as col2)] as myArrayOfStruct,
  PARSE_JSON(inputJSON) as myJSON
FROM INPUTS
"""
job = bq.query(query)
job.result()
job.ended - job.started

datetime.timedelta(seconds=1, microseconds=438000)

In [11]:
query = f"""
SELECT * FROM {project_id}.{dataset_id}.{table_id}_1
"""
example = bq.query(query).to_dataframe()
example

,myInt,myBytes,myFloat,myString,myBool,myGeo,myDate,myDatetime,myTime,myTimestamp,myArray,myStruct,myStructOfStruct,myStructOfArray,myArrayOfStruct,myJSON
0,7,b'stringhere',7.1,string,True,POINT(81.3124 29.9012),2022-10-24,2022-10-24 14:57:07.934451,14:57:07.934451,2022-10-24 14:57:07.934451+00:00,"[1, 2, 3]","{'col1': 1, 'col2': 2, 'col3': 3}","{'myStructNestedA': {'col1': 'a', 'col2': 'b',...","{'myArrayNestedA': [1, 2, 3], 'myArrayNestedB'...","[{'col1': 'a', 'col2': 'B'}, {'col1': '1', 'co...","{""customer"":[{""age"":35,""name"":""Mike""},{""age"":3..."


In [12]:
example.dtypes

myInt                             int64
myBytes                          object
myFloat                         float64
myString                         object
myBool                             bool
myGeo                            object
myDate                           object
myDatetime               datetime64[ns]
myTime                           object
myTimestamp         datetime64[ns, UTC]
myArray                          object
myStruct                         object
myStructOfStruct                 object
myStructOfArray                  object
myArrayOfStruct                  object
myJSON                           object
dtype: object

### Arrays
- like a list, [1, 2, 3, ...]
- repeating values of the same type

In [13]:
myArray = example['myArray'].iloc[0]
myArray

[1, 2, 3]

In [14]:
type(myArray)

list

In [15]:
bq.query(query = f'SELECT myArray FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,myArray
0,"[1, 2, 3]"


In [16]:
bq.query(query = f'SELECT myArray[OFFSET(0)] FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,f0_
0,1


In [17]:
bq.query(query = f'SELECT myArray[ORDINAL(1)] FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,f0_
0,1


In [18]:
bq.query(query = f'SELECT myArray, unnested FROM {project_id}.{dataset_id}.{table_id}_1, UNNEST(myArray) as unnested').to_dataframe()

,myArray,unnested
0,"[1, 2, 3]",1
1,"[1, 2, 3]",2
2,"[1, 2, 3]",3


In [19]:
bq.query(query = f'SELECT myArray, unnested FROM {project_id}.{dataset_id}.{table_id}_1 CROSS JOIN UNNEST(myArray) as unnested').to_dataframe()

,myArray,unnested
0,"[1, 2, 3]",1
1,"[1, 2, 3]",2
2,"[1, 2, 3]",3


In [20]:
bq.query(query = f'SELECT TO_JSON(myArray)[1] FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,f0_
0,2


### Structs
- like a dictionary
- a container of variables, each with their own type

In [21]:
myStruct = example['myStruct'].iloc[0]
myStruct

{'col1': 1, 'col2': 2, 'col3': 3}

In [22]:
type(myStruct)

dict

In [23]:
bq.query(query = f'SELECT myStruct.col2 FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,col2
0,2


In [24]:
bq.query(query = f'SELECT TO_JSON(myStruct).col1 FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,col1
0,1


### Nesting: Struct of Arrays

In [25]:
myStructOfArray = example['myStructOfArray'].iloc[0]
myStructOfArray

{'myArrayNestedA': [1, 2, 3], 'myArrayNestedB': ['A', 'B', 'C', 'D']}

In [26]:
type(myStructOfArray)

dict

In [27]:
type(myStructOfArray['myArrayNestedA'])

list

In [28]:
bq.query(query = f'SELECT myStructOfArray, unnested FROM {project_id}.{dataset_id}.{table_id}_1, UNNEST(myStructOfArray.myArrayNestedA) as unnested').to_dataframe()

,myStructOfArray,unnested
0,"{'myArrayNestedA': [1, 2, 3], 'myArrayNestedB'...",1
1,"{'myArrayNestedA': [1, 2, 3], 'myArrayNestedB'...",2
2,"{'myArrayNestedA': [1, 2, 3], 'myArrayNestedB'...",3


In [29]:
bq.query(query = f'SELECT TO_JSON(myStructOfArray).myArrayNestedA[0] FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,f0_
0,1


### Nesting: Struct of Struct
- a dictionary of dictionaries, like JSON

In [30]:
myStructOfStruct = example['myStructOfStruct'].iloc[0]
myStructOfStruct

{'myStructNestedA': {'col1': 'a', 'col2': 'b', 'col3': 'c'},
 'myStructNestedB': {'col1': 1, 'col2': 'b'}}

In [31]:
type(myStructOfStruct)

dict

In [32]:
type(myStructOfStruct['myStructNestedA'])

dict

In [33]:
bq.query(query = f'SELECT myStructOfStruct.myStructNestedA.col1 FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,col1
0,a


In [34]:
bq.query(query = f'SELECT TO_JSON(myStructOfStruct).myStructNestedA.col1 FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,col1
0,"""a"""


### Nesting: Array of Structs
- a list of dictionaries

In [35]:
myArrayOfStruct = example['myArrayOfStruct'].iloc[0]
myArrayOfStruct

[{'col1': 'a', 'col2': 'B'}, {'col1': '1', 'col2': 'b'}]

In [36]:
type(myArrayOfStruct)

list

In [37]:
type(myArrayOfStruct[0])

dict

In [38]:
bq.query(query = f'SELECT myInt, myArrayOfStruct, unnested, unnested.col1 FROM {project_id}.{dataset_id}.{table_id}_1, UNNEST(myArrayOfStruct) as unnested').to_dataframe()

,myInt,myArrayOfStruct,unnested,col1
0,7,"[{'col1': 'a', 'col2': 'B'}, {'col1': '1', 'co...","{'col1': 'a', 'col2': 'B'}",a
1,7,"[{'col1': 'a', 'col2': 'B'}, {'col1': '1', 'co...","{'col1': '1', 'col2': 'b'}",1


In [39]:
bq.query(query = f'SELECT TO_JSON(myArrayOfStruct)[0].col1 FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,col1
0,"""a"""


### JSON
- dictionaries and lists

In [ ]:
myJSON = example['myJSON'].iloc[0]
myJSON

'{"customer":[{"age":35,"name":"Mike"},{"age":36,"name":"Alice"}]}'

In [ ]:
type(myJSON)

str

In [ ]:
myJSON =  json.loads(myJSON)
myJSON

{'customer': [{'age': 35, 'name': 'Mike'}, {'age': 36, 'name': 'Alice'}]}

In [ ]:
type(myJSON)

dict

In [ ]:
type(myJSON['customer'])

list

In [ ]:
bq.query(query = f'SELECT myJSON.customer FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,customer
0,"[{""age"":35,""name"":""Mike""},{""age"":36,""name"":""Al..."


In [ ]:
bq.query(query = f'SELECT myJSON.customer[0] FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,f0_
0,"{""age"":35,""name"":""Mike""}"


In [ ]:
bq.query(query = f'SELECT myJSON.customer[0].name FROM {project_id}.{dataset_id}.{table_id}_1').to_dataframe()

,name
0,"""Mike"""


---
# NOTES
- Show safe versions of math functions and safe_cast
- detailed use of data types
- for arrays of stucts, show that the nested struct need the same shape and **type**